In [ ]:
import pandas as pd

In [ ]:
import pandas as pd

df = pd.read_csv("/content/smile-annotations-final.xls")   # really CSV but wrong extension
df.to_csv("smile-annotation-final.csv", index=False)


Exploratory data analysis

In [ ]:
import torch
from tqdm.notebook import tqdm

In [ ]:


df = pd.read_csv('smile-annotation-final.csv',
                names=['id', 'text', 'category'])
df.set_index('id', inplace=True)



In [ ]:


df.head()



,text,category
id,,
611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merc...,nocode
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
614877582664835073,@Sofabsports thank you for following me back. ...,happy
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy


In [ ]:
df.category.value_counts()

,count
category,
nocode,1572
happy,1137
not-relevant,214
angry,57
surprise,35
sad,32
happy|surprise,11
happy|sad,9
disgust|angry,7


In [ ]:
df = df[df.category.isin(['happy', 'not-relevant', 'angry', 'surprise', 'sad', 'disgust'])]

In [ ]:
df.category.value_counts()

,count
category,
happy,1137
not-relevant,214
angry,57
surprise,35
sad,32
disgust,6


In [ ]:
possible_labels = df.category.unique()

In [ ]:
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [ ]:
label_dict

{'happy': 0,
 'not-relevant': 1,
 'angry': 2,
 'disgust': 3,
 'sad': 4,
 'surprise': 5}

In [ ]:
df.category = df['category'].map(label_dict)

In [ ]:
df.head()

,text,category
id,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,0
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,0
614877582664835073,@Sofabsports thank you for following me back. ...,0
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,0
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,0


Training/Validation Set

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(df.index.values,
                                                  df.category.values,
                                                  test_size=0.15,
                                                  random_state=42,
                                                  stratify=df.category.values)

In [ ]:
df['data_type'] = ['not_set']*df.shape[0]

In [ ]:
df.head()

,text,category,data_type
id,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,0,not_set
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,0,not_set
614877582664835073,@Sofabsports thank you for following me back. ...,0,not_set
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,0,not_set
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,0,not_set


In [ ]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [ ]:
df.groupby(['category', 'data_type']).count()

text
category data_type      
0        train       966
         val         171
1        train       182
         val          32
2        train        48
         val           9
3        train         5
         val           1
4        train        27
         val           5
5        train        30
         val           5


Loading Tokenizer and Encoding our Data


In [ ]:
!pip install transformers==4.30.0

  Using cached transformers-4.30.0-py3-none-any.whl.metadata (113 kB)
  Using cached tokenizers-0.13.3.tar.gz (314 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Using cached transformers-4.30.0-py3-none-any.whl (7.2 MB)
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Failed to build tokenizers
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (tokenizers)


In [ ]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [ ]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case=True
)

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    padding="max_length",       # ✅ new syntax
    truncation=True,            # ✅ explicitly enable truncation
    max_length=256,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    padding="max_length",       # ✅
    truncation=True,            # ✅
    max_length=256,
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].category.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].category.values)


In [ ]:
dataset_train = TensorDataset(input_ids_train,
                              attention_masks_train,
                              labels_train)

dataset_val = TensorDataset(input_ids_val,
                            attention_masks_val,
                           labels_val)

In [ ]:
len(dataset_train)

1258

In [ ]:
dataset_val.tensors

(tensor([[  101,  1030,  8223,  ...,     0,     0,     0],
         [  101, 13970, 12269,  ...,     0,     0,     0],
         [  101,  1030,  2358,  ...,     0,     0,     0],
         ...,
         [  101,  2023,  2003,  ...,     0,     0,     0],
         [  101,  1030,  2329,  ...,     0,     0,     0],
         [  101,  2204,  2000,  ...,     0,     0,     0]]),
 tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 tensor([1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 2, 4, 0, 1, 0, 1, 0,
         0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 4, 0, 0,
         1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 1, 0, 0, 0, 0, 1, 0, 4, 0, 2, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 2, 1, 1, 0, 3, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,


Setting up BERT Pretrained Model


In [ ]:
from transformers import BertForSequenceClassification

In [ ]:
model = BertForSequenceClassification.from_pretrained(
                                      'bert-base-uncased',
                                      num_labels = len(label_dict),
                                      output_attentions = False,
                                      output_hidden_states = False
                                     )

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Creating Data Loaders

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [ ]:
batch_size = 4

dataloader_train = DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=batch_size
)

dataloader_val = DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=32
)

Setting Up Optimizer and Scheduler

In [ ]:
!pip install transformers==4.30.0

  Using cached transformers-4.30.0-py3-none-any.whl.metadata (113 kB)
  Using cached tokenizers-0.13.3.tar.gz (314 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Using cached transformers-4.30.0-py3-none-any.whl (7.2 MB)
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Failed to build tokenizers
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (tokenizers)


In [ ]:
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup


In [ ]:
optimizer = AdamW(
    model.parameters(),
    lr = 1e-5,
    eps = 1e-8
)

In [ ]:
epochs = 10

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps = len(dataloader_train)*epochs
)


Defining our Performance Metrics

In [ ]:
import numpy as np
from sklearn.metrics import f1_score

In [ ]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average = 'weighted')

In [ ]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}

    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy:{len(y_preds[y_preds==label])}/{len(y_true)}\n')

Creating our Training Loop

In [ ]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

cuda


In [ ]:
def evaluate(dataloader_val):

    model.eval()

    loss_val_total = 0
    predictions, true_vals = [], []

    for batch in tqdm(dataloader_val):

        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():
            outputs = model(**inputs)

        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)

    loss_val_avg = loss_val_total/len(dataloader_val)

    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)

    return loss_val_avg, predictions, true_vals

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    model.train()
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train,
                        desc='Epoch {:1d}'.format(epoch),
                        leave=False,
                        disable=False)

    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2]
        }

        outputs = model(**inputs)
        loss = outputs[0]
        loss_train_total +=loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})

    #torch.save(model.state_dict(), f'Models/BERT_ft_Epoch{epoch}.model')

    tqdm.write('\nEpoch {epoch}')

    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')

    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (weighted): {val_f1}')

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/315 [00:00<?, ?it/s]


Epoch {epoch}
Training loss: 0.7864792277888646


  0%|          | 0/7 [00:00<?, ?it/s]

Validation loss: 0.6001778755869184
F1 Score (weighted): 0.7774699174940259


...

Validation loss: 0.6173505229609353
F1 Score (weighted): 0.88202398130079


Epoch 10:   0%|          | 0/315 [00:00<?, ?it/s]


Epoch {epoch}
Training loss: 0.01599600344207064


  0%|          | 0/7 [00:00<?, ?it/s]

Validation loss: 0.6300201884337834
F1 Score (weighted): 0.8822162646336179


Evaluating our Model

In [ ]:
accuracy_per_class(predictions, true_vals)

Class: happy
Accuracy:163/171

Class: not-relevant
Accuracy:26/32

Class: angry
Accuracy:4/9

Class: disgust
Accuracy:0/1

Class: sad
Accuracy:1/5

Class: surprise
Accuracy:4/5



In [ ]:
output_dir = "./saved_model"

# Save model and tokenizer
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print("Model saved in:", output_dir)


Model saved in: ./saved_model


In [ ]:


!pip install -q gradio


In [ ]:
!pip install --upgrade gradio

In [ ]:
import torch

#weights and bias file
torch.save({
    'model_state_dict': model.state_dict(),
}, "/content/fine_tuned_model.pth")
#from google.colab import files #downloading weights
#files.download("/content/fine_tuned_model.pth")

In [ ]:
import gradio as gr
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import os

# Since you have a .pth file, we need to load it differently
model_path = "/content/fine_tuned_model.pth"

# Check if file exists
if not os.path.exists(model_path):
    raise FileNotFoundError(f"Model file not found at: {model_path}")

print(f"Model file found: {model_path}")

# Load the checkpoint and inspect its contents
checkpoint = torch.load(model_path, map_location='cpu')

print(f"Checkpoint type: {type(checkpoint)}")
if isinstance(checkpoint, dict):
    print(f"Checkpoint keys: {list(checkpoint.keys())}")

# Create the model architecture first
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=6  # Adjust based on your number of classes
)

# Load the state dict from the checkpoint
try:
    if isinstance(checkpoint, dict):
        # Try different common keys for the state dict
        if 'model_state_dict' in checkpoint:
            model.load_state_dict(checkpoint['model_state_dict'])
            print("Loaded model from 'model_state_dict' key")
        elif 'state_dict' in checkpoint:
            model.load_state_dict(checkpoint['state_dict'])
            print("Loaded model from 'state_dict' key")
        elif 'model' in checkpoint:
            model.load_state_dict(checkpoint['model'])
            print("Loaded model from 'model' key")
        else:
            # If none of the common keys exist, try to use the checkpoint directly
            model.load_state_dict(checkpoint)
            print("Loaded model directly from checkpoint")
    else:
        # If checkpoint is not a dict, it might be the state dict itself
        model.load_state_dict(checkpoint)
        print("Loaded model from checkpoint (not a dict)")

    print("Model loaded successfully!")

except Exception as e:
    print(f"Error loading state dict: {e}")
    print("The checkpoint structure might be different. Let's inspect it more:")

    if isinstance(checkpoint, dict):
        for key, value in checkpoint.items():
            print(f"Key: {key}, Type: {type(value)}, Shape: {getattr(value, 'shape', 'N/A')}")

    # You might need to adjust the loading based on the actual structure
    raise RuntimeError("Could not load the model. Please check the checkpoint structure.")

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Set model to evaluation mode
model.eval()

# Define class names - update these to match your training labels
class_names = ['happy', 'not-relevant', 'angry', 'surprise', 'sad', 'disgust']

# Define the sentiment analysis function
def analyze_sentiment(text):
    try:
        # Tokenize the input text
        inputs = tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            padding=True,
            max_length=512
        )

        # Make prediction
        with torch.no_grad():
            outputs = model(**inputs)

        # Get predicted class
        predicted_class = torch.argmax(outputs.logits, dim=-1).item()
        sentiment = class_names[predicted_class]

        # Get confidence scores
        probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
        confidence = probabilities[0][predicted_class].item()

        return sentiment, confidence

    except Exception as e:
        return f"Error: {str(e)}", 0.0

# Define the Gradio interface function
def gradio_app(text):
    if not text.strip():
        return "Please enter some text to analyze."

    sentiment, confidence = analyze_sentiment(text)

    if isinstance(sentiment, str) and sentiment.startswith("Error"):
        return sentiment

    return f"**Sentiment:** {sentiment}\n**Confidence:** {confidence:.2%}"

# Create the Gradio interface
iface = gr.Interface(
    fn=gradio_app,
    inputs=gr.Textbox(
        placeholder="Type your sentence here...",
        label="Input Text",
        lines=3
    ),
    outputs=gr.Textbox(label="Analysis Result"),
    title="Sentiment Analysis with Fine-tuned BERT",
    description="Enter a sentence to analyze its sentiment using your fine-tuned BERT model.",
    examples=[
        ["I'm so happy today! This is amazing!"],
        ["This movie is terrible and boring."],
        ["What a surprise! I didn't expect this."],
        ["I feel really sad about this situation."],
        ["This food tastes disgusting."],
        ["The weather forecast for tomorrow."]
    ]
)

# Launch the Gradio app
print("Launching Gradio interface...")
iface.launch(share=True, debug=True)

# Optional: Display model information
print(f"\nModel Information:")
print(f"- Model type: {type(model).__name__}")
print(f"- Number of parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"- Number of classes: {len(class_names)}")
print(f"- Class names: {class_names}")

# Alternative loading method if the above doesn't work:
"""
# If you need to load with specific configurations:

# Method 3: Custom loading with explicit config
from transformers import BertConfig

config = BertConfig.from_pretrained('bert-base-uncased')
config.num_labels = 6  # Your number of classes

model = BertForSequenceClassification(config)
checkpoint = torch.load('/content/fine_tuned_model.pth', map_location='cpu')

# Try different key patterns
if 'model' in checkpoint:
    model.load_state_dict(checkpoint['model'])
elif 'net' in checkpoint:
    model.load_state_dict(checkpoint['net'])
else:
    model.load_state_dict(checkpoint)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
"""

Model file found: /content/fine_tuned_model.pth
Checkpoint type: <class 'dict'>
Checkpoint keys: ['model_state_dict']


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Loaded model from 'model_state_dict' key
Model loaded successfully!
Launching Gradio interface...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://5c41637f609ed45767.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
